In [ ]:
# Start H2O-3 runtime.

import h2o
h2o.init(strict_version_check=False)

In [ ]:
# Configure DRF algorithm and train a model.

from h2o.estimators import H2ORandomForestEstimator

# Import the cars dataset into H2O:
cars = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")

# Set the predictors and response;
# set the response as a factor:
cars["economy_20mpg"] = cars["economy_20mpg"].asfactor()
predictors = ["displacement","power","weight","acceleration","year"]
response = "economy_20mpg"

# Split the dataset into a train and valid set:
train, valid = cars.split_frame(ratios=[.8], seed=1234)
drf = H2ORandomForestEstimator(ntrees=10,
                                    max_depth=5,
                                    min_rows=10,
                                    calibrate_model=True,
                                    calibration_frame=valid,
                                    binomial_double_trees=True)
drf.train(x=predictors,
          y=response,
          training_frame=train,
          validation_frame=valid)

In [ ]:
# Log the model to an MLFlow reqistry.

import mlflow
import h2o_mlflow_flavor
mlflow.set_tracking_uri("http://127.0.0.1:8080")

with mlflow.start_run(run_name="cars") as run:
    mlflow.log_params(h2o_mlflow_flavor.get_params(drf)) # Log training parameters of the model (optional).
    mlflow.log_metrics(h2o_mlflow_flavor.get_metrics(drf)) # Log performance matrics of the model (optional).
    input_example = h2o_mlflow_flavor.get_input_example(drf) # Extract input example from training dataset (optional)
    h2o_mlflow_flavor.log_model(drf, "cars", input_example=input_example,
                                model_type="MOJO", # Specify whether the output model should be MOJO or POJO. (MOJO is default)
                                extra_prediction_args=["--predictCalibrated"]) # Add extra prediction args if needed.

In [ ]:
# Load model from the MLFlow registry and score with the model.

import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:8080")

logged_model = 'runs:/a9ff364f07fa499eb44e7c49e47fab11/cars' # Specify correct id of your run.

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
data = pd.read_csv("https://s3.amazonaws.com/h2o-public-test-data/smalldata/junit/cars_20mpg.csv")
loaded_model.predict(data)